# OOP (2024 Fall) HW1: 불변 순서쌍을 활용해 가변 순서쌍 만들기 
- 이름: 유지현
- 학번: 20222281

여기 홍길동, 99999999 대신 본인의 이름, 학번 작성

-----
## Part A: 불변 순서쌍 Pair
1. 클래스(또는 정적) 팩토리 메소드 `of`를 를 수정하여 완성하라.
    - https://velog.io/@cjh8746/%EC%A0%95%EC%A0%81-%ED%8C%A9%ED%86%A0%EB%A6%AC-%EB%A9%94%EC%84%9C%EB%93%9CStatic-Factory-Method
1. 주석 부분을 풀어 제너릭 클래스인 `Pair`가 제너릭 인터페이스 `Comparable`를 구현하도록 완성하라.
    - https://docs.oracle.com/en/java/javase/22/docs/api/java.base/java/lang/Comparable.html

In [2]:
record Pair<F extends Comparable<F>, S extends Comparable<S>>(F first, S second) implements Comparable<Pair<F, S>>  /* implements Comparable< ... > */ {
    
    // 아래 팩토리 메소드 of가 컴파일되지 않고 있으니 컴파일되도록 수정하라 (힌트: 제너릭 메소드)
    static <F extends Comparable<F>, S extends Comparable<S>> Pair<F, S> of(F first, S second) {
        return new Pair<>(first, second);
    }
    
    // 사전순이 되도록 정의하기. 즉 first가 더 큰 쪽이 더 크고, first가 같으면 second를 비교
    @Override
    public int compareTo(Pair<F, S> other) {
        int firstCompare = this.first.compareTo(other.first);
        if (firstCompare != 0) {
            return firstCompare;
        }
        return this.second.compareTo(other.second);
    }
}

In [9]:
/* compareTo 가 잘 동작하는지
   - 왼쪽이 더 큰 경우
   - 왼쪽이 더 작은 경우
   - 같은 경우
   이런 경우를 각각 포함하도록 이렇게 최소 3개 이상의 테스트를 실행해 보라
*/
// 왼쪽 값이 더 큰 경우
Pair<Integer, String> pair1 = Pair.of(3, "apple");
Pair<Integer, String> pair2 = Pair.of(1, "banana");

// 왼쪽 값이 더 작은 경우
Pair<Integer, String> pair3 = Pair.of(2, "orange");
Pair<Integer, String> pair4 = Pair.of(5, "grape");

// 값이 같은 경우
Pair<Integer, String> pair5 = Pair.of(4, "mango");
Pair<Integer, String> pair6 = Pair.of(4, "mango");

// 서로 다른 타입 (Integer, Double)
Pair<Integer, Double> pair11 = Pair.of(4, 5.5);
Pair<Integer, Double> pair12 = Pair.of(4, 3.14);


// 대략 이런 식으로 테스트 실행
System.out.println(pair1.compareTo(pair2)); // 결과는 양수 (3 > 1)
System.out.println(pair3.compareTo(pair4)); // 결과는 음수 (2 < 5)
System.out.println(pair5.compareTo(pair6));
System.out.println(pair11.compareTo(pair12)); // 결과는 양수 (5.5 > 3.14)
// System.out.println( ??.compareTo(??) );
// System.out.println( ??.compareTo(??) );
// System.out.println( ??.compareTo(??) );

1
-1
0
1


In [12]:
class ModifiablePair<F extends Comparable<F>, S extends Comparable<S>> implements Comparable<ModifiablePair<F, S>> /* implements Comparable< ... > */ {
    private Pair<F,S> pair; // 이 field 말고 다른 field는 추가로 정의하지 말 것!

    ModifiablePair(F first, S second) {
        this.pair = new Pair<>(first, second);
    }
    
    // getter들은 pair에게 위임(delegate)하는 방식으로 작성하라
    public F first() {
        return pair.first();
    }
    public S second() {
        return pair.second();
    }
    
    // setter들은 적절히 정의하라
    public void setFirst(F first) {
        this.pair = new Pair<>(first, this.pair.second());
    }
    public void setSecond(S second) {
        this.pair = new Pair<>(this.pair.first(), second);
    }

    
    // compareTo는 pair에게 위임(delegate)하는 방식으로 작성하라
    @Override
    public int compareTo(ModifiablePair<F, S> other) {
        return this.pair.compareTo(other.pair);
    }
    

    @Override
    public String toString() { 
        return pair.toString(); // toString은 그냥 pair에게 위임
    }
}

In [15]:
// ModifiablePair<F,S>의 getter와 setter들이 잘 동작하는지 확인할 수 있는 적절한 테스트 코드 작성하여 실행하라.
// 그러니까 `first()`, `second()`, `setFrist(F)`, `setSecond(S)` 메소드를 모두 활용하는 예시 코드를 작성해 실행하라는 말이다.

// ModifiablePair 생성
ModifiablePair<Integer, String> pair = new ModifiablePair<>(1, "apple");

// 초기 값 확인
System.out.println("Initial values: ");
System.out.println("First: " + pair.first());   // 출력: First: 1
System.out.println("Second: " + pair.second()); // 출력: Second: apple

// First 값 수정
pair.setFirst(5);
System.out.println("\nAfter setting first to 5: ");
System.out.println("First: " + pair.first());   // 출력: First: 5
System.out.println("Second: " + pair.second()); // 출력: Second: apple

// Second 값 수정
pair.setSecond("orange");
System.out.println("\nAfter setting second to 'orange': ");
System.out.println("First: " + pair.first());   // 출력: First: 5
System.out.println("Second: " + pair.second()); // 출력: Second: orange

// 두 ModifiablePair 객체 비교
ModifiablePair<Integer, String> pair2 = new ModifiablePair<>(5, "banana");
System.out.println("\nComparing pair with pair2: ");
System.out.println(pair.compareTo(pair2)); // pair의 second가 "orange"이고 pair2는 "banana"이므로 양수

// toString 테스트
System.out.println("\nPair toString: ");
System.out.println(pair); // 출력: Pair(5, orange)


Initial values: 
First: 1
Second: apple

After setting first to 5: 
First: 5
Second: apple

After setting second to 'orange': 
First: 5
Second: orange

Comparing pair with pair2: 
13

Pair toString: 
Pair[first=5, second=orange]
